Using Drive

In [ ]:
!pip install pydrive

In [59]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

# Authenticate with Google Drive
gauth = GoogleAuth()
gauth.LocalWebserverAuth() # Authenticate using a local web server
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=93210286347-n059g7t2kfn9ute2csnsmbog59gksuvd.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code



In [38]:
import os
from tqdm import tqdm 
import json

def get_directories():
    folder_id = '1Agx0XbceiVg4lgYX09hbyySvkWWVRWI1'
    file_list = drive.ListFile({'q': f"'{folder_id}' in parents"}).GetList()
    dir = [(f['id'], f['title']) for f in file_list if f['mimeType'].endswith('folder')]
    return dir

def get_images(folder_id):
    file_list = drive.ListFile({'q': f"'{folder_id}' in parents"}).GetList()
    image_files = [file for file in file_list if file['title'].lower().endswith(('.jpg', '.jpeg', '.png', '.gif'))]
    result = []
    for img in image_files:
        if 'cover' in img['title']:
            result.insert(0,{
            'img': img['webContentLink'],
            'alt': 'None',
            'location': None,
            'tag': None
        })
        else:
            result.append({
                'img': img['webContentLink'],
                'alt': 'None',
                'location': None,
                'tag': None
            })
    return result

def run():
    config = []
    dirs = get_directories()
    for path, title in tqdm(dirs):
        photos = get_images(path)
        if len(photos):
            config.append({
                    'title': title.title(),
                    'img': photos[0]['img'],
                    'photos': photos
            })
    return config

config = run()


json.dump(config, open('../src/data/image_config.json', 'w'))

100%|██████████| 2/2 [00:01<00:00,  1.69it/s]


Git code

In [56]:
import requests
import base64
import json
import datetime


def push_to_repo_branch(gitHubFileName, fileName, repo_slug, branch, user, token):
    '''
    Push file update to GitHub repo
    
    :param gitHubFileName: the name of the file in the repo
    :param fileName: the name of the file on the local branch
    :param repo_slug: the github repo slug, i.e. username/repo
    :param branch: the name of the branch to push the file to
    :param user: github username
    :param token: github user token
    :return None
    :raises Exception: if file with the specified name cannot be found in the repo
    '''
    
    message = "Automated update " + str(datetime.datetime.now())
    path = "https://api.github.com/repos/%s/branches/%s" % (repo_slug, branch)

    r = requests.get(path, auth=(user,token))
    if not r.ok:
        print("Error when retrieving branch info from %s" % path)
        print("Reason: %s [%d]" % (r.text, r.status_code))
        raise
    rjson = r.json()
    treeurl = rjson['commit']['commit']['tree']['url']
    r2 = requests.get(treeurl, auth=(user,token))
    if not r2.ok:
        print("Error when retrieving commit tree from %s" % treeurl)
        print("Reason: %s [%d]" % (r2.text, r2.status_code))
        raise
    r2json = r2.json()
    sha = None

    for file in r2json['tree']:
        # Found file, get the sha code
        if file['path'] == gitHubFileName:
            sha = file['sha']

    # # if sha is None after the for loop, we did not find the file name!
    # if sha is None:
    #     print ("Could not find " + gitHubFileName + " in repos 'tree' ")
    #     raise Exception

    with open(fileName, 'rb') as data:
        content = base64.b64encode(data.read()).decode()

    # gathered all the data, now let's push
    inputdata = {}
    inputdata["path"] = gitHubFileName
    inputdata["branch"] = branch
    inputdata["message"] = message
    inputdata["content"] = content
    if sha:
        inputdata["sha"] = str(sha)

    updateURL = f"https://api.github.com/repos/{repo_slug}/contents/" + gitHubFileName
    try:
        rPut = requests.put(updateURL, auth=(user,token), data = json.dumps(inputdata))
        if not rPut.ok:
            print("Error when pushing to %s" % updateURL)
            print("Reason: %s [%d]" % (rPut.text, rPut.status_code))
            raise Exception
    except requests.exceptions.RequestException as e:
        print ('Something went wrong! I will print all the information that is available so you can figure out what happend!')
        print (rPut)
        print (rPut.headers)
        print (rPut.text)
        print (e)

github_username = 'Sarita-Joshi'
repo_name = 'photography-portfolio'
access_token = ''


push_to_repo_branch(
    'src/pages/image_config.json',
    '../src/data/image_config.json',
    'Sarita-Joshi/web-portfolio',
    'develop',
    github_username,
    access_token
)
